# The idea of permutation

This page shows how permutation works by comparing to a physical
implementation of permutation, that randomizes values by mixing balls in a
bucket.


## Example: do mosquitoes like beer?

This page shows how permutation works by comparing to a physical implementation
of permutation by mixing balls in a bucket.

![](images/mosquito_banner.png)

With thanks to John Rauser: [Statistics Without the Agonizing
Pain](https://www.youtube.com/watch?v=5Dnw46eC-0o)


## The data

If you want to run this notebook on your own computer, Download the data from
{download}`mosquito_beer.csv <data/mosquito_beer.csv>`.

See [this
page](https://github.com/matthew-brett/datasets/tree/master/mosquito_beer) for
more details on the dataset, and [the data license page](data/LICENSE).


In [ ]:
# Import Numpy library, rename as "np"
import numpy as np
# Make random number generator.
rng = np.random.default_rng()
# Import Pandas library, rename as "pd"
import pandas as pd
# Safe setting for Pandas.
pd.set_option('mode.copy_on_write', True)

# Set up plotting
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
# HIDDEN
# An extra tweak to make sure we always get the same random numbers.
# We need the same set of random numbers to match the illustrations below.
# Do not use this in your own code; you nearly always want an unpredictable
# stream of random numbers.  Making them predictable in this way only makes
# sense for a very limited range of things, like tutorials and tests.
# As you can see, we replace the usual random number generator we
# made above, with another one with predictable random numbers.
rng = np.random.default_rng(seed=42)

Read in the data:

In [ ]:
mosquitoes = pd.read_csv('data/mosquito_beer.csv')
mosquitoes.head()

Filter the data frame to contain only the "after" treatment rows:

In [ ]:
# After treatment rows.
afters = mosquitoes[mosquitoes['test'] == 'after']

Filter the "after" rows to contain only the "beer" group, and get the number of activated mosquitoes for these 25 subjects:

In [ ]:
# After beer treatment rows.
beers = afters[afters['group'] == 'beer']
# The 'activated' numbers for the after beer rows.
beer_activated = np.array(beers['activated'])
beer_activated

The number of subjects in the "beer" condition:

In [ ]:
n_beer = len(beer_activated)
n_beer

Get the "activated" number for the 18 subjects in the "water" group:

In [ ]:
# Same for the water group.
waters = afters[afters['group'] == 'water']
water_activated = np.array(waters['activated'])
water_activated

Number of subjects in the "water" condition:

In [ ]:
n_water = len(water_activated)
n_water

## The permutation way

* Calculate difference in means
* Pool
* Repeat many times:
    * Shuffle (permute)
    * Split
    * Recalculate difference in means
    * Store

The next graphic shows the activated values as a series of gold and blue
balls.  The activated numbers for the "beer" group are gold), and the activated
numbers for the "water" group, in blue:

![](images/just_balls.png)

## Calculate difference in means

Here we take the mean of "beer" activated numbers (the numbers in gold):

![](images/beer_mean.png)

In [ ]:
beer_mean = np.mean(beer_activated)
beer_mean

Next we take the mean of activation values for the "water" subjects (value in
blue):

![](images/water_mean.png)

In [ ]:
water_mean = np.mean(water_activated)
water_mean

The difference between the means in our data:

In [ ]:
observed_difference = beer_mean - water_mean
observed_difference

## Pool

We can put the values values for the beer and water conditions into one long
array, 25 + 18 values long.

In order to do this, we use the `np.concatenate` function.  It does what we
want; it takes two arrays and splices them together into one long array.  This
operation is called *concatenation*.

Here is `np.concatenate` in action:

In [ ]:
first_array = np.array([10, 20, 30])
second_array = np.array([99, 199, 299])
# The two arrays concatenated.
both_together = np.concatenate([first_array, second_array])
both_together

We apply `np.concatenate` to *pool* our two groups of numbers into one array.

In [ ]:
pooled = np.concatenate([beer_activated, water_activated])
pooled

## Shuffle

Then we shuffle the pooled values so the beer and water values are completely
mixed.

In [ ]:
shuffled = rng.permutation(pooled)
shuffled

This is the same idea as putting the gold and blue balls into a bucket and shaking them up into a random arrangement.

![](images/fake_balls0.png)

## Split

We take the first 25 values as our fake beer group.  In fact these 25 values
are a random mixture of the beer and the water values.  This is the same idea as taking 25 balls at random from the jumbled mix of gold and blue balls.

In [ ]:
# Take the first 25 values
fake_beer = shuffled[:n_beer]

![](images/fake_beer_mean0.png)

We calculate the mean:

In [ ]:
fake_beer_mean = np.mean(fake_beer)
fake_beer_mean

Then we take the remaining 18 values as our fake water group:

In [ ]:
fake_water = shuffled[n_beer:]

![](images/fake_water_mean0.png)

We take the mean of these too:

In [ ]:
fake_water_mean = np.mean(fake_water)
fake_water_mean

The difference between these means is our first estimate of how much the mean difference will vary when we take random samples from this pooled population:

In [ ]:
fake_diff = fake_beer_mean - fake_water_mean
fake_diff

## Repeat

We do another shuffle:

In [ ]:
shuffled = rng.permutation(pooled)

![](images/fake_beer_mean1.png)

We take another fake beer group, and calculate another fake beer mean:

In [ ]:
fake_beer = shuffled[:n_beer]
np.mean(fake_beer)

We take another fake water group, find the mean:

![](images/fake_water_mean1.png)

In [ ]:
fake_water = shuffled[n_beer:]
np.mean(fake_water)

Now we have another example difference between these means:

In [ ]:
np.mean(fake_beer) - np.mean(fake_water)

We can keep on repeating this process to get more and more examples of mean
differences:

In [ ]:
# Shuffle
shuffled = rng.permutation(pooled)
# Split
fake_beer = shuffled[:n_beer]
fake_water = shuffled[n_beer:]
# Recalculate mean difference
fake_diff = np.mean(fake_beer) - np.mean(fake_water)
fake_diff

It is not hard to do this as many times as we want, using a `for` loop:

In [ ]:
fake_differences = np.zeros(10000)
for i in np.arange(10000):
    # Shuffle
    shuffled = rng.permutation(pooled)
    # Split
    fake_beer = shuffled[:n_beer]
    fake_water = shuffled[n_beer:]
    # Recalculate mean difference
    fake_diff = np.mean(fake_beer) - np.mean(fake_water)
    # Store mean difference
    fake_differences[i] = fake_diff
plt.hist(fake_differences);

We are interested to know just how unusual it is to get a difference as big as we actually see, in these many samples of differences we expect by chance, from random sampling.

To do this we calculate how many of the fake differences we generated are equal to or greater than the difference we observe:

In [ ]:
n_ge_actual = np.count_nonzero(fake_differences >= observed_difference)
n_ge_actual

That means that the chance of any one difference being greater than the one we observe is:

In [ ]:
p_ge_actual = n_ge_actual / 10000
p_ge_actual

This is also an estimate of the probability we would see a difference as large as the one we observe, if we were taking random samples from a matching population.